Project Scenario:
---------------------------------------

You have been hired as a data engineer by research organization. Your boss has asked you to create a code that can be used to compile the list of the top 10 largest banks in the world ranked by market capitalization in billion USD. Further, the data needs to be transformed and stored in GBP, EUR and INR as well, in accordance with the exchange rate information that has been made available to you as a CSV file. The processed information table is to be saved locally in a CSV format and as a database table.

Your job is to create an automated system to generate this information so that the same can be executed in every financial quarter to prepare the report.


Particulars of the code to be made have been shared below.
---------------------------------------

Data URL	https://web.archive.org/web/20230908091635 /https://en.wikipedia.org/wiki/List_of_largest_banks

Exchange rate CSV path	https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv

Table Attributes (upon Extraction only)	Name, MC_USD_Billion

Table Attributes (final)	Name, MC_USD_Billion, MC_GBP_Billion, MC_EUR_Billion, MC_INR_Billion

Output CSV Path	./Largest_banks_data.csv

Database name	Banks.db

Table name	Largest_banks

Log file	code_log.txt


Project tasks
---------------------------------------

Task 1:
Write a function log_progress() to log the progress of the code at different stages in a file code_log.txt. Use the list of log points provided to create log entries as every stage of the code.

Task 2:
Extract the tabular information from the given URL under the heading 'By market capitalization' and save it to a dataframe.
a. Inspect the webpage and identify the position and pattern of the tabular information in the HTML code
b. Write the code for a function extract() to perform the required data extraction.
c. Execute a function call to extract() to verify the output.

Task 3:
Transform the dataframe by adding columns for Market Capitalization in GBP, EUR and INR, rounded to 2 decimal places, based on the exchange rate information shared as a CSV file.
a. Write the code for a function transform() to perform the said task.
b. Execute a function call to transform() and verify the output.

Task 4:
Load the transformed dataframe to an output CSV file. Write a function load_to_csv(), execute a function call and verify the output.

Task 5:
Load the transformed dataframe to an SQL database server as a table. Write a function load_to_db(), execute a function call and verify the output.

Task 6:
Run queries on the database table. Write a function load_to_db(), execute a given set of queries and verify the output.

Task 7:
Verify that the log entries have been completed at all stages by checking the contents of the file code_log.txt.

In [10]:
# Code for ETL operations on Top 10 banks by market capitilization data


# Importing required libraries

import pandas as pd
import numpy as np
import requests
import sqlite3
from bs4 import BeautifulSoup
from datetime import datetime

# URLs
marketcapdataurl = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
exchangerateurl = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv'

# Output locations
table_name = 'Largest_banks'
db_name = 'Banks.db'
output_csv_path = './Largest_banks_data.csv'


# Log Function

def log_progress(message):
    ''' This function logs the mentioned message at a given stage of the 
    code execution to a log file. Function returns nothing.'''
    
    timestamp_format = '%Y-%b-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open("./code_log.txt", "a") as f:
        f.write(timestamp + ' : ' + message + '\n')


log_progress('Preliminaries complete. Initiating ETL process')

# Extract Function
    
def extract(marketcapdataurl):
    '''Extract the list of top banks and their market capitalization in USD from the website.'''
    page = requests.get(marketcapdataurl).text
    data = BeautifulSoup(page, 'html.parser')
    df = pd.DataFrame(columns=['Name', 'MC_USD_Billion'])
    
    # Get the table from the webpage
    tables = data.find_all('tbody')
    rows = tables[0].find_all('tr')  # Ensure you're accessing the correct table
    
    for row in rows:
        col = row.find_all('td')
        if len(col) >= 3 and col[1].find('a') is not None and '—' not in col[2].get_text():
            try:
                name = col[1].get_text().strip()
                mc_value = col[2].get_text().replace(',', '').strip()
                mc_value = float(mc_value)  # Convert to float for further calculations
                df = pd.concat([df, pd.DataFrame({'Name': [name], 'MC_USD_Billion': [mc_value]})], ignore_index=True)
            except Exception as e:
                log_progress(f"Skipping a row due to error: {e}")
                continue
    return df

log_progress('Data extraction complete. Initiating Transformation process')

df = extract(marketcapdataurl)

# Transform Function

def transform(df):
    
    '''Extract exchange rates from the CSV file and return them as a dictionary.
    Convert the market cap of banks to GBP, EUR, and INR using exchange rates.'''

    
    exchange_rate = pd.read_csv('exchange_rate.csv') #Reading the exchange rate file into a dataframe
    exchange_rate_dict = dict(zip(exchange_rate['Currency'],exchange_rate['Rate'])) #Converting dataframe to dict
    
    #Adding the new columns based on currency rates
    
    df['MC_GBP_Billion'] = [np.round(x*exchange_rate_dict['GBP'],2) for x in df['MC_USD_Billion']]
    df['MC_EUR_Billion'] = [np.round(x*exchange_rate_dict['EUR'],2) for x in df['MC_USD_Billion']]
    df['MC_INR_Billion'] = [np.round(x*exchange_rate_dict['INR'],2) for x in df['MC_USD_Billion']]

    return df
        
df = transform(df)

log_progress('Data Transformation complete. Initiating loading process')

# Load Function

def load_to_csv(df):
    
    '''Save the transformed data to CSV. Function returns nothing'''
    
    df.to_csv(output_csv_path, index=False)
    return None

load_to_csv(df)
log_progress('Data saved to CSV file')

# Initiating SQL Connection
    
sql_connection = sqlite3.connect(db_name)
log_progress('SQL connection initiated')

def load_to_db(df):
    
    '''Save the transformed data to CSV. Function returns nothing'''

    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)
    return None

load_to_db(df)
log_progress('Data loaded to Database as a table, Executing queries')


#SQL Query Function

def run_query(query_statement, sql_connection):
    ''' This function runs the stated query on the database table and
    prints the output on the terminal. Function returns nothing. '''

    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)

#Running SQL queries for business
    
query_statement = f"SELECT * from {table_name}"
run_query(query_statement, sql_connection)

query_statement = f"SELECT AVG(MC_GBP_Billion) FROM  {table_name}"
run_query(query_statement, sql_connection)

query_statement = f"SELECT Name FROM  {table_name} LIMIT 5"
run_query(query_statement, sql_connection)

log_progress('Process Complete.')

sql_connection.close()

log_progress('Server connection closed')


SELECT * from Largest_banks
                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of China          160.68          128.54   
4                                HDFC Bank          157.91          126.33   
5                              Wells Fargo          155.87          124.70   
6                        HSBC Holdings PLC          148.90          119.12   
7                           Morgan Stanley          140.83          112.66   
8                  China Construction Bank          139.82          111.86   
9                            Bank of China          136.81          109.45   

   MC_EUR_Billion  MC_INR_Billion  
0          402.62        35910.71  
1          215.31        19204.58  
2    